## 📌 Задача 9: Декартово дерево (Treap) (Региональный этап ВСОШ 2023)

**Условие:** Реализовать структуру данных, поддерживающую следующие операции:
1. Вставить число x
2. Удалить число x
3. Найти k-й по величине элемент
4. Найти количество элементов в диапазоне [l, r]

**Ограничения:** 1 ≤ Q ≤ 2·10⁵ операций

**Решение:** Декартово дерево (Treap) с поддержкой размеров поддеревьев и неявными ключами.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

typedef long long ll;
mt19937 rnd(chrono::steady_clock::now().time_since_epoch().count());

struct Node {
    int key, priority, size;
    Node *left, *right;
    Node(int k) : key(k), priority(rnd()), size(1), left(nullptr), right(nullptr) {}
};

int get_size(Node* node) {
    return node ? node->size : 0;
}

void update_size(Node* node) {
    if (node) {
        node->size = 1 + get_size(node->left) + get_size(node->right);
    }
}

void split(Node* root, int key, Node*& left, Node*& right) {
    if (!root) {
        left = right = nullptr;
        return;
    }
    if (root->key <= key) {
        split(root->right, key, root->right, right);
        left = root;
    } else {
        split(root->left, key, left, root->left);
        right = root;
    }
    update_size(root);
}

Node* merge(Node* left, Node* right) {
    if (!left || !right) return left ? left : right;
    if (left->priority > right->priority) {
        left->right = merge(left->right, right);
        update_size(left);
        return left;
    } else {
        right->left = merge(left, right->left);
        update_size(right);
        return right;
    }
}

Node* insert(Node* root, Node* node) {
    if (!root) return node;
    if (node->priority > root->priority) {
        split(root, node->key, node->left, node->right);
        update_size(node);
        return node;
    }
    if (node->key < root->key) {
        root->left = insert(root->left, node);
    } else {
        root->right = insert(root->right, node);
    }
    update_size(root);
    return root;
}

Node* erase(Node* root, int key) {
    if (!root) return nullptr;
    if (root->key == key) {
        Node* result = merge(root->left, root->right);
        delete root;
        return result;
    }
    if (key < root->key) {
        root->left = erase(root->left, key);
    } else {
        root->right = erase(root->right, key);
    }
    update_size(root);
    return root;
}

int kth_element(Node* root, int k) {
    if (!root) return -1;
    int left_size = get_size(root->left);
    if (k <= left_size) {
        return kth_element(root->left, k);
    } else if (k == left_size + 1) {
        return root->key;
    } else {
        return kth_element(root->right, k - left_size - 1);
    }
}

int count_range(Node* root, int l, int r) {
    if (!root) return 0;
    if (root->key < l) {
        return count_range(root->right, l, r);
    } else if (root->key > r) {
        return count_range(root->left, l, r);
    } else {
        return 1 + count_range(root->left, l, r) + count_range(root->right, l, r);
    }
}

void solve() {
    Node* root = nullptr;
    int q;
    cin >> q;
    
    while (q--) {
        int type, x;
        cin >> type >> x;
        
        if (type == 1) {
            root = insert(root, new Node(x));
        } else if (type == 2) {
            root = erase(root, x);
        } else if (type == 3) {
            cout << kth_element(root, x) << '\n';
        } else if (type == 4) {
            int y;
            cin >> y;
            cout << count_range(root, x, y) << '\n';
        }
    }
}

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(nullptr);
    solve();
    return 0;
}

## 📌 Задача 10: Алгоритм Флойда-Уоршелла с восстановлением пути (Муниципальный этап ВСОШ 2024)

**Условие:** Дан ориентированный граф с N вершинами и M ребрами. Каждое ребро имеет вес. Найти длину кратчайшего пути между всеми парами вершин и вывести матрицу расстояний. Если пути не существует, вывести -1.

**Ограничения:** 1 ≤ N ≤ 500, 1 ≤ M ≤ N(N-1), веса ребер до 10⁹

**Решение:** Алгоритм Флойда-Уоршелла с оптимизацией и восстановлением путей для небольших N.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

typedef long long ll;
const ll INF = 1e18;
const int MAXN = 505;

ll dist[MAXN][MAXN];
int next_node[MAXN][MAXN];

void solve() {
    int n, m;
    cin >> n >> m;
    
    // Инициализация
    for (int i = 1; i <= n; i++) {
        for (int j = 1; j <= n; j++) {
            dist[i][j] = INF;
            next_node[i][j] = -1;
            if (i == j) dist[i][j] = 0;
        }
    }
    
    // Чтение ребер
    for (int i = 0; i < m; i++) {
        int u, v;
        ll w;
        cin >> u >> v >> w;
        if (w < dist[u][v]) {
            dist[u][v] = w;
            next_node[u][v] = v;
        }
    }
    
    // Алгоритм Флойда-Уоршелла
    for (int k = 1; k <= n; k++) {
        for (int i = 1; i <= n; i++) {
            for (int j = 1; j <= n; j++) {
                if (dist[i][k] < INF && dist[k][j] < INF && 
                    dist[i][j] > dist[i][k] + dist[k][j]) {
                    dist[i][j] = dist[i][k] + dist[k][j];
                    next_node[i][j] = next_node[i][k];
                }
            }
        }
    }
    
    // Вывод матрицы расстояний
    for (int i = 1; i <= n; i++) {
        for (int j = 1; j <= n; j++) {
            if (dist[i][j] == INF) {
                cout << "-1 ";
            } else {
                cout << dist[i][j] << " ";
            }
        }
        cout << "\n";
    }
    
    // Пример восстановления пути между двумя вершинами
    int start, end;
    cin >> start >> end;
    
    if (dist[start][end] == INF) {
        cout << "No path" << endl;
        return;
    }
    
    vector<int> path;
    int cur = start;
    path.push_back(cur);
    
    while (cur != end && next_node[cur][end] != -1) {
        cur = next_node[cur][end];
        path.push_back(cur);
    }
    
    cout << "Path length: " << dist[start][end] << endl;
    cout << "Path: ";
    for (int node : path) {
        cout << node << " ";
    }
    cout << endl;
}

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(nullptr);
    solve();
    return 0;
}

## 📌 Задача 11: Алгоритм Краскала с DSU (Региональный этап ВСОШ 2024)

**Условие:** Дан неориентированный взвешенный граф. Найти вес минимального остовного дерева (MST). Если граф несвязный, вывести -1.

**Ограничения:** 1 ≤ N ≤ 10⁵, 1 ≤ M ≤ 2·10⁵, веса ребер до 10⁹

**Решение:** Алгоритм Краскала с системой непересекающихся множеств (DSU) для эффективного построения MST.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

typedef long long ll;
typedef pair<ll, pair<int, int>> edge;

struct DSU {
    vector<int> parent, rank;
    DSU(int n) {
        parent.resize(n + 1);
        rank.resize(n + 1, 0);
        for (int i = 1; i <= n; i++) parent[i] = i;
    }
    
    int find(int x) {
        if (parent[x] != x) {
            parent[x] = find(parent[x]);
        }
        return parent[x];
    }
    
    bool unite(int x, int y) {
        x = find(x), y = find(y);
        if (x == y) return false;
        if (rank[x] < rank[y]) swap(x, y);
        parent[y] = x;
        if (rank[x] == rank[y]) rank[x]++;
        return true;
    }
};

void solve() {
    int n, m;
    cin >> n >> m;
    
    vector<edge> edges;
    for (int i = 0; i < m; i++) {
        int u, v;
        ll w;
        cin >> u >> v >> w;
        edges.push_back({w, {u, v}});
    }
    
    // Сортировка ребер по весу
    sort(edges.begin(), edges.end());
    
    DSU dsu(n);
    ll mst_weight = 0;
    int edges_used = 0;
    
    for (auto& e : edges) {
        ll w = e.first;
        int u = e.second.first;
        int v = e.second.second;
        
        if (dsu.unite(u, v)) {
            mst_weight += w;
            edges_used++;
            if (edges_used == n - 1) break;
        }
    }
    
    if (edges_used != n - 1) {
        cout << "-1" << endl;
    } else {
        cout << mst_weight << endl;
    }
}

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(nullptr);
    solve();
    return 0;
}

## 📌 Задача 12: Быстрая сортировка с подсчетом инверсий (Муниципальный этап ВСОШ 2023)

**Условие:** Дан массив из N чисел. Найти количество инверсий в массиве (пар индексов (i, j), где i < j и a[i] > a[j]).

**Ограничения:** 1 ≤ N ≤ 10⁶, |a[i]| ≤ 10⁹

**Решение:** Модификация сортировки слиянием для подсчета инверсий за O(N log N).

In [ ]:
#include <bits/stdc++.h>
using namespace std;

typedef long long ll;
const int MAXN = 1e6 + 5;

int a[MAXN], temp[MAXN];
ll inversions = 0;

void merge(int l, int mid, int r) {
    int i = l, j = mid + 1, k = l;
    
    while (i <= mid && j <= r) {
        if (a[i] <= a[j]) {
            temp[k++] = a[i++];
        } else {
            temp[k++] = a[j++];
            inversions += (mid - i + 1); // Все оставшиеся элементы в левой части образуют инверсии
        }
    }
    
    while (i <= mid) {
        temp[k++] = a[i++];
    }
    
    while (j <= r) {
        temp[k++] = a[j++];
    }
    
    for (int i = l; i <= r; i++) {
        a[i] = temp[i];
    }
}

void merge_sort(int l, int r) {
    if (l >= r) return;
    int mid = (l + r) / 2;
    merge_sort(l, mid);
    merge_sort(mid + 1, r);
    merge(l, mid, r);
}

void solve() {
    int n;
    cin >> n;
    
    for (int i = 0; i < n; i++) {
        cin >> a[i];
    }
    
    inversions = 0;
    merge_sort(0, n - 1);
    
    cout << inversions << endl;
}

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(nullptr);
    solve();
    return 0;
}

## 📌 Задача 13: Алгоритм Ахо-Корасик для поиска нескольких шаблонов (Региональный этап ВСОШ 2023)

**Условие:** Дан текст T и набор шаблонов P₁, P₂, ..., Pₖ. Найти общее количество вхождений всех шаблонов в текст.

**Ограничения:** |T| ≤ 10⁶, суммарная длина всех шаблонов ≤ 10⁶

**Решение:** Алгоритм Ахо-Корасик для эффективного поиска нескольких шаблонов одновременно.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

const int ALPHABET = 26;
const int MAXN = 1e6 + 5;

struct Node {
    int next[ALPHABET];
    int fail_link;
    int output;
    int count;
    
    Node() {
        memset(next, -1, sizeof(next));
        fail_link = -1;
        output = 0;
        count = 0;
    }
};

vector<Node> trie;

void add_string(const string& s) {
    int v = 0;
    for (char c : s) {
        int idx = c - 'a';
        if (trie[v].next[idx] == -1) {
            trie[v].next[idx] = trie.size();
            trie.push_back(Node());
        }
        v = trie[v].next[idx];
    }
    trie[v].output++;
}

void build_automaton() {
    queue<int> q;
    
    // Инициализация для корня
    for (int c = 0; c < ALPHABET; c++) {
        if (trie[0].next[c] != -1) {
            int v = trie[0].next[c];
            trie[v].fail_link = 0;
            q.push(v);
        } else {
            trie[0].next[c] = 0;
        }
    }
    
    // BFS для построения fail-ссылок
    while (!q.empty()) {
        int v = q.front();
        q.pop();
        
        for (int c = 0; c < ALPHABET; c++) {
            int u = trie[v].next[c];
            if (u != -1) {
                trie[u].fail_link = trie[trie[v].fail_link].next[c];
                trie[u].output += trie[trie[u].fail_link].output;
                q.push(u);
            } else {
                trie[v].next[c] = trie[trie[v].fail_link].next[c];
            }
        }
    }
}

ll count_occurrences(const string& text) {
    int v = 0;
    ll total = 0;
    
    for (char c : text) {
        int idx = c - 'a';
        v = trie[v].next[idx];
        total += trie[v].output;
    }
    
    return total;
}

void solve() {
    trie.clear();
    trie.push_back(Node()); // Корень
    
    int k;
    cin >> k;
    
    for (int i = 0; i < k; i++) {
        string pattern;
        cin >> pattern;
        add_string(pattern);
    }
    
    build_automaton();
    
    string text;
    cin >> text;
    
    cout << count_occurrences(text) << endl;
}

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(nullptr);
    solve();
    return 0;
}

## 📌 Задача 14: Динамическое программирование с оптимизацией разделяй и властвуй (Региональный этап ВСОШ 2024)

**Условие:** Дан массив из N чисел. Разбить массив на K непрерывных подмассивов так, чтобы сумма квадратов сумм элементов в каждом подмассиве была минимальной.

**Ограничения:** 1 ≤ N ≤ 10⁵, 1 ≤ K ≤ min(N, 100)

**Решение:** ДП с оптимизацией разделяй и властвуй (Divide and Conquer Optimization) для решения за O(K·N·log N).

In [ ]:
#include <bits/stdc++.h>
using namespace std;

typedef long long ll;
const ll INF = 1e18;
const int MAXN = 1e5 + 5;
const int MAXK = 105;

ll a[MAXN], prefix[MAXN];
ll dp[MAXK][MAXN];
int opt[MAXK][MAXN];

// Вспомогательная функция для вычисления стоимости подмассива [l, r]
ll cost(int l, int r) {
    ll sum = prefix[r] - prefix[l - 1];
    return sum * sum;
}

void compute(int k, int l, int r, int opt_l, int opt_r) {
    if (l > r) return;
    
    int mid = (l + r) / 2;
    dp[k][mid] = INF;
    
    // Оптимальная точка разбиения лежит между opt_l и min(opt_r, mid-1)
    int opt_mid = opt_l;
    for (int i = opt_l; i <= min(opt_r, mid - 1); i++) {
        ll current = dp[k - 1][i] + cost(i + 1, mid);
        if (current < dp[k][mid]) {
            dp[k][mid] = current;
            opt_mid = i;
        }
    }
    
    opt[k][mid] = opt_mid;
    
    // Рекурсивно вычисляем для левой и правой частей
    compute(k, l, mid - 1, opt_l, opt_mid);
    compute(k, mid + 1, r, opt_mid, opt_r);
}

void solve() {
    int n, k;
    cin >> n >> k;
    
    for (int i = 1; i <= n; i++) {
        cin >> a[i];
        prefix[i] = prefix[i - 1] + a[i];
    }
    
    // Базовый случай: k = 1
    for (int i = 1; i <= n; i++) {
        dp[1][i] = cost(1, i);
        opt[1][i] = 0;
    }
    
    // Динамика для k >= 2
    for (int current_k = 2; current_k <= k; current_k++) {
        compute(current_k, current_k, n, current_k - 1, n - 1);
    }
    
    cout << dp[k][n] << endl;
}

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(nullptr);
    solve();
    return 0;
}

## 📌 Задача 15: Алгоритм Диница для максимального потока (Региональный этап ВСОШ 2023)

**Условие:** Дан ориентированный граф с пропускными способностями ребер. Найти максимальный поток из источника S в сток T.

**Ограничения:** 1 ≤ N ≤ 1000, 1 ≤ M ≤ 10000, пропускные способности до 10⁹

**Решение:** Алгоритм Диница с уровневыми графами и блокирующими потоками для эффективного вычисления максимального потока.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

typedef long long ll;
const ll INF = 1e18;
const int MAXN = 1005;
const int MAXM = 20005;

struct Edge {
    int to, rev;
    ll cap;
};

vector<Edge> g[MAXN];
int level[MAXN], ptr[MAXN];
int n, m, s, t;

void add_edge(int u, int v, ll cap) {
    g[u].push_back({v, (int)g[v].size(), cap});
    g[v].push_back({u, (int)g[u].size() - 1, 0});
}

bool bfs() {
    memset(level, -1, sizeof(level));
    queue<int> q;
    q.push(s);
    level[s] = 0;
    
    while (!q.empty()) {
        int v = q.front();
        q.pop();
        
        for (auto& e : g[v]) {
            if (e.cap > 0 && level[e.to] == -1) {
                level[e.to] = level[v] + 1;
                q.push(e.to);
            }
        }
    }
    
    return level[t] != -1;
}

ll dfs(int v, ll pushed) {
    if (pushed == 0) return 0;
    if (v == t) return pushed;
    
    for (int& cid = ptr[v]; cid < g[v].size(); cid++) {
        Edge& e = g[v][cid];
        
        if (e.cap > 0 && level[e.to] == level[v] + 1) {
            ll tr = dfs(e.to, min(pushed, e.cap));
            if (tr > 0) {
                e.cap -= tr;
                g[e.to][e.rev].cap += tr;
                return tr;
            }
        }
    }
    
    return 0;
}

ll dinic() {
    ll flow = 0;
    while (bfs()) {
        memset(ptr, 0, sizeof(ptr));
        while (ll pushed = dfs(s, INF)) {
            flow += pushed;
        }
    }
    return flow;
}

void solve() {
    cin >> n >> m >> s >> t;
    
    for (int i = 0; i < m; i++) {
        int u, v;
        ll cap;
        cin >> u >> v >> cap;
        add_edge(u, v, cap);
    }
    
    cout << dinic() << endl;
}

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(nullptr);
    solve();
    return 0;
}

## 📌 Задача 16: Выпуклая оболочка методом Джарвиса (Муниципальный этап ВСОШ 2024)

**Условие:** Дано N точек на плоскости. Найти площадь выпуклой оболочки этих точек.

**Ограничения:** 1 ≤ N ≤ 10⁵, координаты точек до 10⁹

**Решение:** Алгоритм Джарвиса (метод заворачивания подарка) для построения выпуклой оболочки и вычисление площади.

In [ ]:
#include <bits/stdc++.h>
using namespace std;

typedef long long ll;
const int MAXN = 1e5 + 5;

struct Point {
    ll x, y;
    
    Point() : x(0), y(0) {}
    Point(ll x, ll y) : x(x), y(y) {}
    
    Point operator-(const Point& other) const {
        return Point(x - other.x, y - other.y);
    }
    
    ll cross(const Point& other) const {
        return x * other.y - y * other.x;
    }
    
    ll dot(const Point& other) const {
        return x * other.x + y * other.y;
    }
    
    bool operator<(const Point& other) const {
        return x < other.x || (x == other.x && y < other.y);
    }
};

Point points[MAXN];

ll cross(const Point& a, const Point& b, const Point& c) {
    return (b - a).cross(c - a);
}

vector<Point> convex_hull_jarvis(int n) {
    vector<Point> hull;
    if (n == 0) return hull;
    
    // Находим самую левую нижнюю точку
    int start = 0;
    for (int i = 1; i < n; i++) {
        if (points[i].x < points[start].x || 
            (points[i].x == points[start].x && points[i].y < points[start].y)) {
            start = i;
        }
    }
    
    int current = start;
    do {
        hull.push_back(points[current]);
        int next = (current + 1) % n;
        
        for (int i = 0; i < n; i++) {
            if (i == current) continue;
            ll cross_val = cross(points[current], points[next], points[i]);
            if (cross_val > 0 || 
                (cross_val == 0 && (points[i] - points[current]).dot(points[i] - points[current]) > 
                (points[next] - points[current]).dot(points[next] - points[current]))) {
                next = i;
            }
        }
        
        current = next;
    } while (current != start);
    
    return hull;
}

ll polygon_area(const vector<Point>& polygon) {
    int n = polygon.size();
    if (n < 3) return 0;
    
    ll area = 0;
    for (int i = 0; i < n; i++) {
        int j = (i + 1) % n;
        area += polygon[i].x * polygon[j].y - polygon[j].x * polygon[i].y;
    }
    
    return abs(area);
}

void solve() {
    int n;
    cin >> n;
    
    for (int i = 0; i < n; i++) {
        cin >> points[i].x >> points[i].y;
    }
    
    vector<Point> hull = convex_hull_jarvis(n);
    ll area = polygon_area(hull);
    
    // Площадь с точностью до 0.5
    cout << area / 2;
    if (area % 2 == 1) cout << ".5";
    cout << endl;
}

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(nullptr);
    solve();
    return 0;
}